In [2]:
import numpy as np
import matplotlib.pyplot as plt
from pyts.image import RecurrencePlot
from scipy.spatial.distance import pdist, squareform

from data import get_lorenz_data
#from data_Chen import get_chen_data
#from data_Chua import get_chua_data
#from data_Liu import get_liu_data
#from data_R import get_rossler_data


##    help function      ##
def rec_plot(s, eps=0.10, steps=10):
    d = pdist(s[:,None])
    d = np.floor(d/eps)
    d[d>steps] = steps
    Z = squareform(d)
    return Z

def moving_average(s, r=5):
    return np.convolve(s, np.ones((r,))/r, mode='valid')

def distance_matrix(data, dimension, delay, norm):
    N = int(len(data) - (dimension-1) * delay)
    distance_matrix = np.zeros((N, N), dtype="float32")
    if norm == 'manhattan':
        for i in range(N):
            for j in range(i, N, 1):
                temp = 0.0
                for k in range(dimension):
                    temp += np.abs(data[i+k*delay] - data[j+k*delay])
                distance_matrix[i,j] = distance_matrix[j,i] = temp
    elif norm == 'euclidean':
        for i in range(N):
            for j in range(i, N, 1):
                temp = 0.0
                for k in range(dimension):
                    temp += np.power(data[i+k*delay] - data[j+k*delay], 2)
                distance_matrix[i,j] = distance_matrix[j,i] = np.sqrt(temp)
    elif norm == 'supremum':
        temp = np.zeros(dimension)
        for i in range(N):
            for j in range(i, N, 1):
                for k in range(dimension):
                    temp[k] = np.abs(data[i+k*delay] - data[j+k*delay])
                distance_matrix[i,j] = distance_matrix[j,i] = np.max(temp)
    return distance_matrix

def recurrence_matrix(data, dimension, delay, threshold, norm):
    recurrence_matrix = distance_matrix(data, dimension, delay, norm)
    N = len(recurrence_matrix[:,0])
    for i in range(N):
        for j in range(i, N, 1):
            if recurrence_matrix[i,j] <= threshold:
                recurrence_matrix[i,j] = recurrence_matrix[j,i] = 1
            else:
                recurrence_matrix[i,j] = recurrence_matrix[j,i] = 0
    return recurrence_matrix.astype(int)
##########################
if __name__=="__main__":
    rho = np.arange(28, 60, 0.032)
    for i in rho:
        dt = 0.02
        traj = get_lorenz_data(dt=dt, rho=i)
        #train_data, val_data = get_chen_data(dt=dt)
        #train_data, val_data = get_chua_data(dt=dt)
        #train_data, val_data = get_liu_data(dt=dt)
        #train_data, val_data = get_rossler_data(dt=dt)
        eps, steps = 1.0, 3
        X = traj[:1000,0]
        dimension, delay, threshold, norm = 3, 2, 7, "manhattan"
        #distance_matrix = distance_matrix(X, dimension, delay, norm)
        #recurrence_matrix = recurrence_matrix(X, dimension, delay, threshold, norm)
        #LIN = len(distance_matrix[:,0])

        plt.figure(num=None, figsize=(7,7))
        plt.subplots_adjust(top=1, bottom=0, left=0, right=1, hspace=0, wspace=0)
        plt.imshow(distance_matrix(X, dimension, delay, norm),
                   cmap =  'binary',origin='lower')
        #plt.axis([-0.5, LIN-0.5, -0.5, LIN-0.5])
        #plt.xticks([x for x in range(LIN)], [x+1 for x in range(LIN)])
        #plt.yticks([x for x in range(LIN)], [x+1 for x in range(LIN)])
        #plt.colorbar(fraction=0.046, pad=0.04)
        plt.savefig(r'./pic_lor/lor_'+str(i)+'.png',dpi=32)
        plt.close()
        plt.show()